In [ ]:
import os
import shutil
import pandas as pd
import geopandas as gpd
import rasterio
import s2cloudless
import numpy as np
import rasterio.warp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import mysecrets
import fetch_from_cluster
import modify_images
import rsutils.utils
import create_stack
import stack_ops

In [ ]:
zip_filepath = fetch_from_cluster.download_file_from_cluster(
    sshcreds = mysecrets.SSH_UMD_SASIRAJANN,
    remotepath = '/gpfs/data1/cmongp2/sasirajann/data/misc/165bca4_s2l1c.zip',
    download_folderpath = '../data/cluster_files',
    # overwrite = True,
)

In [ ]:
stack_folderpath = '../data/165bca4_s2l1c_stack'

In [ ]:
shutil.unpack_archive(zip_filepath, stack_folderpath)

In [ ]:
bands, metadata = create_stack.load_stack(stack_folderpath)

In [ ]:
metadata['bands'].remove('CMK')
metadata['bands']

In [ ]:
bands[:,:,:,:-1].shape

In [ ]:
bands, metadata = stack_ops.run_s2cloudless(bands[:,:,:,:-1], metadata=metadata, chunksize=1)

In [ ]:
bands.shape, metadata['bands'], metadata['geotiff_metadata']